In [35]:
import pandas as pd
import numpy as np
data = pd.read_csv('./elektrokonverzija_obradjeno.csv', index_col=0)
print(data.columns)

Index(['stariji od 65', 'Pol', 'Datum elektrokonverzije',
       'indikacija: perzistentna',
       'ukupno trajanje AF (kada je prvi put dijagnostikovana) u mesecima',
       'trajanje ove epizode AF u mesecima',
       'frekvenca komora na prijemnom EKG-u', 'Blok grane', 'dimenzija LPK',
       'EDD LK', 'ESD LK', 'EF LK', 'MR', 'TR', 'HTA', 'DM', 'HLP', 'HOBP',
       'pušenje', 'Hereditet za KVB', 'ishemijska bolest srca', 'HF',
       'NYHA klasa (ukoliko ima HF)', 'Prethodni TIA/CVI', 'HBI', 'CHADS-Vasc',
       'poremećaj f-je štitaste žlezde', 'dosadašnje elektrokonverzije',
       'SCD (Structural heart disease)', 'Beta blokatori', 'ACE inhibitori',
       'ARB', 'Furosemid (ne - 0)', 'Drugi diuretik', 'Spironolakton',
       'Ca antagonisti', 'Amiodaron', 'Propafen', 'Flekainid', 'Sotalol',
       'Digoksin', 'ASA', 'OAKT', 'Plavix', 'Statin', 'Implantiran CIED',
       'Vreme u mesecima od ugradnje aparata do elektrokonverzije',
       'sinhroni, bifazni DC šok', 'Elektrokon

In [36]:
from pgmpy.models import BayesianModel

kolone_za_predikciju = data.columns[:-1]
kolona_predikcije = data.columns[-1]

model = BayesianModel([(kolona_predikcije, att) for att in kolone_za_predikciju])

from pgmpy.estimators import BayesianEstimator, MaximumLikelihoodEstimator

model.fit(
    data=data,
    estimator=BayesianEstimator,
    prior_type="BDeu",
    equivalent_sample_size=100,
    complete_samples_only=False
)

print(model.get_cpds()[2])

+---------------------------------------------+--------------------------------------------------+--------------------------------------------------+
| Održavanje sinusnog ritma nakon godinu dana | Održavanje sinusnog ritma nakon godinu dana(0.0) | Održavanje sinusnog ritma nakon godinu dana(1.0) |
+---------------------------------------------+--------------------------------------------------+--------------------------------------------------+
| Pol(1)                                      | 0.6153846153846154                               | 0.66                                             |
+---------------------------------------------+--------------------------------------------------+--------------------------------------------------+
| Pol(2)                                      | 0.38461538461538464                              | 0.34                                             |
+---------------------------------------------+--------------------------------------------------+--

In [37]:
from pgmpy.inference import VariableElimination
inference = VariableElimination(model)

y = []
y_pred = []

for index, row in data.iterrows():
    y.append(row[kolona_predikcije])

    ev = {}
    for att in kolone_za_predikciju:
        ev[att] = data.loc[0, att]
    y_pred.append(inference.map_query([kolona_predikcije], evidence=ev, show_progress=False)[kolona_predikcije])

In [38]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y, y_pred)

array([[ 0, 80],
       [ 0, 50]])